In [12]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx.utils import powerlaw_sequence
from networkx.algorithms.dag import dag_longest_path

# Function 1.Generate graph by Configuration Model

Generate a degree sequence with **nodeN** nodes, and **meanDegree**



In [37]:
def generate_graph(nodeN, mean_degree):
    # Generate degree sequence
    sequence = np.random.poisson(mean_degree, nodeN)
    while (np.sum(sequence) % 2 !=0):
        sequence = np.random.poisson(mean_degree, nodeN)
    # sequence = nx.random_powerlaw_tree_sequence(nodeN, tries=5000)

    # Generate Graph according to the degree sequence
    G = nx.configuration_model(sequence)

    # Remove parallel edges
    G = nx.Graph(G)

    # Remove self-loops
    G.remove_edges_from(nx.selfloop_edges(G))
    return G    

# Function 2. Decide nodes' mask wearing states

## P(A person wears a mask) = m

Generate the mask wearing states of each node.

In [38]:
def init_mask(G, mask_prob):
    # A list of 1 and 0 indicating mask wearing or not
    # 1 means wear mask, 0 means not wearing a mask
    masks = np.random.binomial(1, mask_prob, nodeN)

    # Node idx
    nodes = np.linspace(0, nodeN - 1, nodeN, dtype = int)
    # Dict of node attributes
    mask_dict = dict(zip(nodes, masks))

    # Set nodes attributes
    nx.set_node_attributes(G, mask_dict, 'mask')    
    return G, mask_dict

# Function 3. Init nodes infection states to all 0 (not infected)

In [39]:
def init_infected(G):
    # Init all nodes to be healthy
    infected = np.zeros(nodeN, dtype = int)

    # Node idx
    nodes = np.linspace(0, nodeN - 1, nodeN, dtype = int)

    # Dict of node attributes
    infected_dict = dict(zip(nodes, infected))

    # Set nodes attributes
    nx.set_node_attributes(G, infected_dict, 'infected')
    
    return G, infected_dict   

# Function 4. Generate the BFS tree structures for each components

In [40]:
def generate_BFS_tree(G, mask_dict, infected_dict):
    # Get all the connected components
    components = list(nx.connected_components(G))

    # Roots stores the randomly selected root for each components
    roots = []
    for component in components:
        roots.append(random.choice(list(component)))
        
    # Convert the components to a BFS tree-like structure, with randomly selected roots
    # Trees stores all the tree-structured components
    Trees = []
    for root in roots:
        T = nx.bfs_tree(G, source = root)
        nx.set_node_attributes(T, mask_dict, 'mask')
        nx.set_node_attributes(T, infected_dict, 'infected')
        Trees.append(T)
    
    return roots, Trees

# Function 5. Starting the infection process with 1 virus strain 

**Notice**
Infection starts from root to the leaves

| Infectious    |  Susceptible   | Transmissibillity     | Notation |
| :------------- | :----------: | :----------- | :----------- |
| 1             | 0              | T * T_mask^2 * m      | T_1 |
| 1             | 1              | T * T_mask * (1 - m)  | T_2 |
| 0             | 0              | T * (1 - m)           | T_3 |
| 0             | 1              | T * T_mask * m        | T_4 |




In [41]:
def generate_new_transmissibilities(T_mask, T, m):
    roundN = 5 # Round T to roundN digits
    T1 = round(T * T_mask * T_mask * m, roundN)
    T2 = round(T * T_mask * (1 - m), roundN)
    T3 = round(T * (1 - m), roundN)
    T4 = round(T * T_mask * m , roundN)

    trans_dict = {'T1': T1,
                  'T2': T2,
                  'T3': T3,
                  'T4': T4}

    print("T1: %.5f" %T1)
    print("T2: %.5f" %T2)
    print("T3: %.5f" %T3)
    print("T4: %.5f" %T4)
    
    return trans_dict    

# Function 6: Traverse from the root nodes using bfs search

Starting with 1 seed

In [42]:
def start_infection(Trees, roots, trans_dict, infected_dict):
    for idx, tree in enumerate(Trees): 
#         print("TREE No.", idx)

        edge_attr = dict()
        root = roots[idx]
        dfs_edges = list(nx.bfs_edges(tree, source = root))
        total_depth = nx.dag_longest_path_length(tree)

        nx.set_node_attributes(tree, infected_dict, 'infected')
        nx.set_node_attributes(tree, {root: 1}, 'infected') # Let root node be infected by nature


        for depth in range(1, total_depth + 1): # Transmitted level by level
#             print('LEVEL %d' % depth)

            if depth == 1: # Get only this level's node pairs
                dfs_edges = list(nx.dfs_edges(tree, source = root, depth_limit = depth))

            else: 

                dfs_edges = set(nx.dfs_edges(tree, source = root, depth_limit = depth)) - \
                            set(nx.dfs_edges(tree, source = root, depth_limit = depth - 1))


            for father, son in dfs_edges: # Check each node pairs in this level
#                 print("(%d, %d), node %d is_infected = %d" %(father, son, father, tree.nodes[father]['infected'] ))

                if tree.nodes[father]['infected'] == 1: 

                    # Decide which transmissibility
                    if tree.nodes[father]['mask'] == 1 and tree.nodes[son]['mask'] == 0:
                        T_edge = 'T1'
                    elif tree.nodes[father]['mask'] == 1 and tree.nodes[son]['mask'] == 1:
                        T_edge = 'T2'
                    elif tree.nodes[father]['mask'] == 0 and tree.nodes[son]['mask'] == 0:
                        T_edge = 'T3'
                    else:
                        T_edge = 'T4'

                    edge_attr[(father,son)] = {'T': T_edge}


                    # Set the 'Transmissibility'edge attrs
                    nx.set_edge_attributes(tree, edge_attr)


                    # Decide if the susceptible is infected
                    is_infected = int(random.random() < trans_dict[T_edge])

#                     if is_infected:
#                         print("node %d is infected with %s" %(son, T_edge))
#                     else:
#                         print("node %d is not infected" %(son))


                    # Set the 'infected' node attr accordingly
                    nx.set_node_attributes(tree, {son: is_infected}, 'infected')    

## Look at the infected result

change i to look at different sub-trees

In [43]:
i = 0
pos = nx.nx_agraph.graphviz_layout(Trees[i])

nx.draw(Trees[i], pos, with_labels = False)

# # Show nodes' attrs
node_labels = nx.get_node_attributes(Trees[i],'infected')
nx.draw_networkx_labels(Trees[i], pos, labels = node_labels)
print("root node:", roots[i])
# Show edges' attrs
edge_labels = nx.get_edge_attributes(Trees[i],'T')
nx.draw_networkx_edge_labels(Trees[i], pos, labels = edge_labels)

NameError: name 'Trees' is not defined

# Function 7: Calculate the Epidemic size for a G

In [44]:
def cal_EpdSize_Trees(Trees):
    E_S = []
    for idx, tree  in enumerate(Trees):
        res = np.array(list(tree.nodes.data('infected')))
#         print(res)
        es = sum(res[:,1]) / res.shape[0]
        E_S.append(es)
#         print("Tree %d Epidemic Size: %.3f" %(idx, es))
    avg_ES = sum(E_S) / len(E_S)
    print('Avg Epidemic Size:', avg_ES)
    return avg_ES    

# Function 0: Run the experiment

In [48]:
def runExp_mask(expN, mean_degree, nodeN, mask_prob, T_mask, T, ):
    '''
    Arg: 
    mean_degree: Random graph's mean degree
    nodeN:       Node number
    mask_prob:   P(a person wearing a mask)
    T_mask:      The impact of wearing a mask
    T:           The transmissibility of a real strain
    
    '''
    print("EXP", expN)
    G = generate_graph(nodeN, mean_degree)
    G, mask_dict = init_mask(G, mask_prob)
    G, infected_dict = init_infected(G)
    roots, Trees = generate_BFS_tree(G, mask_dict, infected_dict)
    
    # Transmission with mask
    trans_dict_mask = generate_new_transmissibilities(T_mask, T, mask_prob) #
    start_infection(Trees, roots, trans_dict_mask, infected_dict) #
    cal_EpdSize_Trees(Trees)       

In [49]:
expN = 1
mean_degree = 2
nodeN = 20
mask_prob = 0.6
T_mask = 0.1
T = 0.8

In [50]:
runExp_mask(expN, mean_degree, nodeN, mask_prob, T_mask, T)

EXP 1
T1: 0.00480
T2: 0.03200
T3: 0.32000
T4: 0.04800
Avg Epidemic Size: 0.6851851851851851
